### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re

### Create Base Soup Object - Single Page Run

In [2]:
r = requests.get('https://j-archive.com/showgame.php?game_id=7404')
soup = bs(r.content )

In [3]:
j_category_list = soup.find('div', attrs = {'id':'jeopardy_round'}).find_all('td', attrs = {'class':'category_name'})
j_categories = []
for category in j_category_list:
    j_categories.append(category.get_text())

### Get Show Info

In [4]:
show_info = soup.find('div', attrs = {'id':'game_title'}).text

In [5]:
show_num = re.findall(r'\d+', show_info)[0]
show_yr = re.findall(r'\d+', show_info)[2][2:]

### Identify Daily Doubles

In [6]:
indexer = 1
dd_list = []
for thing in soup.find_all('td', class_ = 'clue'):
    row = {}
    row['q_index'] = indexer
    indexer += 1
    if thing.find('td', class_ = 'clue_value_daily_double'):
        row['is_DD'] = 1
    else:
        row['is_DD'] = 0
    dd_list.append(row)
df_dd = pd.DataFrame(dd_list)

### Identify Answers and Triple Stumpers

In [7]:
answers = []
counter = 1

for clue in soup.find_all('td', class_ = 'clue'):
    answer = {}
    if counter < 61:
        answer['q_index'] = counter
        ans = clue.find('div', onmouseover = True).get('onmouseover')
        cls = bs(ans)
        answer['answer'] = cls.find('em').text
        if 'Triple Stumper' in ans:
            answer['is_stumper'] = 1
        else:
            answer['is_stumper'] = 0
        answers.append(answer)
        counter += 1
df_ans = pd.DataFrame(answers)

### Create Question List

In [8]:
j_value = ['200','400','600','800','1000'] #should these be ints?
dj_value = ['400','800','1200','1600','2000'] #SHould these be ints?

In [9]:
# Jeopardy Categories
j_category_list = soup.find('div', attrs = {'id':'jeopardy_round'}).find_all('td', attrs = {'class':'category_name'})
j_categories = []
for category in j_category_list:
    j_categories.append(category.get_text())

In [10]:
# Jeopardy:
questions = []

for i in range(6):
    for j in range(5):
        question = {}
        
        question['show_num'] = show_num
        question['show_yr'] = show_yr
        
        question['round'] = 'Jeopardy'
        
        question['category'] = j_categories[i]
         
        question['value'] = j_value[j]
        
        clue = f'clue_J_{i+1}_{j+1}' 
        question['clue'] = soup.find('td', attrs = {'id':clue}).text
        
        question['q_index'] = (i + 1) + (6* j)
        
        questions.append(question)
df_j_qs = pd.DataFrame(questions)

In [11]:
# Double Jeopardy Categories
dj_category_list = soup.find('div', attrs = {'id':'double_jeopardy_round'}).find_all('td', attrs = {'class':'category_name'})
dj_categories = []
for category in dj_category_list:
    dj_categories.append(category.get_text())

In [20]:
# Double Jeopardy
questions = []

for i in range(6):
    for j in range(5):
        question = {}
        
        question['show_num'] = show_num
        question['show_yr'] = show_yr
        
        question['round'] = 'Double Jeopardy'
        
        question['category'] = dj_categories[i]
         
        question['value'] = dj_value[j]
        
        clue = f'clue_DJ_{i+1}_{j+1}' 
        question['clue'] = soup.find('td', attrs = {'id':clue}).text
        
        question['q_index'] = 30 + (i + 1) + (6* j)
        
        questions.append(question)
df_dj_qs = pd.DataFrame(questions)

In [17]:
# Final Jeopardy

#q_index is unnecessary as this final jeopardy will be assigned after other dfs are combined

fj = soup.find('table', class_ = 'final_round')

f_j = {}
f_j['show_num'] = show_num
f_j['show_yr'] = show_yr
f_j['round'] = 'Final Jeopardy'
f_j['value'] = 'FJ'
f_j['is_DD'] = 0

f_j['category'] = fj.find('td', class_ = 'category_name').text

f_j['clue'] = fj.find('td', attrs = {'id':'clue_FJ'}).text

ans = fj.find('div', onmouseover = True).get('onmouseover')
cls = bs(ans)
f_j['answer'] = cls.find('em').text
if 'Triple Stumper' in ans:
    f_j['is_stumper'] = 1
else:
    f_j['is_stumper'] = 0

df_fj = pd.DataFrame([f_j])

### Combine Dataframes

In [24]:
df_js = pd.concat([df_j_qs, df_dj_qs]).reset_index()

In [34]:
df_game = df_js.merge(df_dd, on='q_index').merge(df_ans, on = 'q_index').drop(columns=['index','q_index'])
df_game = pd.concat([df_game,df_fj]).reset_index()
df_game

,index,show_num,show_yr,round,category,value,clue,is_DD,answer,is_stumper
0,0,8677,22,Jeopardy,IT HAPPENED IN JULY,200,"On July 20, 1969 he famously declared:""That's ...",0,Armstrong,0
1,1,8677,22,Jeopardy,IT HAPPENED IN JULY,400,"Killed while making bread on July 3, 1863, Jen...",0,(the Battle of) Gettysburg,0
2,2,8677,22,Jeopardy,IT HAPPENED IN JULY,600,Escaping from prison days before he was to be ...,0,Billy the Kid,0
3,3,8677,22,Jeopardy,IT HAPPENED IN JULY,800,"On July 17, 2020 America lost 2 civil rights i...",0,John Lewis,0
4,4,8677,22,Jeopardy,IT HAPPENED IN JULY,1000,"The NHS for short, it got started July 5, 1948...",0,the National Health Service,0
...,...,...,...,...,...,...,...,...,...,...
56,56,8677,22,Double Jeopardy,TOOL WORDS & PHRASES,800,Familiar with the procedure? Then you know thi...,0,a drill,0
57,57,8677,22,Double Jeopardy,TOOL WORDS & PHRASES,1200,It can mean both to sculpt & to swindle someone,0,chisel,1
58,58,8677,22,Double Jeopardy,TOOL WORDS & PHRASES,1600,"To twist, or slang for a prison guard",0,a screw,1
59,59,8677,22,Double Jeopardy,TOOL WORDS & PHRASES,2000,Alliterative phrase meaning to drive a fact in...,0,to hammer home,0


---------

# Functions

In [ ]:
import acquire